In [55]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score, classification_report, confusion_matrix, accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector as SFS, SelectFromModel
from xgboost import XGBClassifier as XGBC
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV

import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import copy

In [56]:
train7=pd.read_csv('Timestep5_train_day7orLess.csv')
test7=pd.read_csv('Timestep5_test_day7orLess.csv')
hold7=pd.read_csv('Timestep5_holdout_day7orLess.csv')

traintest= pd.read_csv('Mmerged_icu_mean.csv')
hold= pd.read_csv('MmergedHoldout.csv')


In [57]:
full=pd.concat([traintest, hold])

full['id'] = full['id'].astype(str)

In [58]:
#get id
train7ID= train7['id'].astype(str)
test7ID= test7['id'].astype(str)
hold7ID= hold7['id'].astype(str)


In [59]:
#split with same id
merged= full.loc[full['id'].isin(train7ID)]
mergedT= full.loc[full['id'].isin(test7ID)]
mergedH= full.loc[full['id'].isin(hold7ID)]


In [60]:
merged['los_bins']=merged.apply(lambda row: 0 if row['los_icu']<=7 else 1 , axis=1 )
mergedT['los_bins']=mergedT.apply(lambda row: 0 if row['los_icu']<=7 else 1 , axis=1 )
mergedH['los_bins']=mergedH.apply(lambda row: 0 if row['los_icu']<=7 else 1 ,axis=1 )

C:\Users\kawai\AppData\Local\Temp\ipykernel_28128\3920080498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['los_bins']=merged.apply(lambda row: 0 if row['los_icu']<=7 else 1 , axis=1 )
C:\Users\kawai\AppData\Local\Temp\ipykernel_28128\3920080498.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergedT['los_bins']=mergedT.apply(lambda row: 0 if row['los_icu']<=7 else 1 , axis=1 )
C:\Users\kawai\AppData\Local\Temp\ipykernel_28128\3920080498.py:3: SettingWithCopyWarning: 
A value is trying to b

In [61]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14281 entries, 1 to 2040
Data columns (total 68 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   id                                                               14281 non-null  object 
 1   hosp_admittime                                                   14281 non-null  object 
 2   hosp_dischtime                                                   14281 non-null  object 
 3   icu_intime                                                       14281 non-null  object 
 4   icu_outtime                                                      14281 non-null  object 
 5   los_icu                                                          14281 non-null  float64
 6   icu_death                                                        14281 non-null  int64  
 7   gender                                   

In [62]:
cla_merged=merged.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_death','los_icu','icu_outcome'], axis=1)
cla_mergedT=mergedT.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_death','los_icu','icu_outcome'],axis=1)
cla_mergedH=mergedH.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_death','los_icu','icu_outcome'],axis=1)

In [63]:
cla_mergedT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4082 entries, 3 to 2039
Data columns (total 60 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   gender                                                           4082 non-null   int64  
 1   admission_age                                                    4082 non-null   float64
 2   weight_admit                                                     4082 non-null   float64
 3   height                                                           4082 non-null   float64
 4   charlson_score                                                   4082 non-null   int64  
 5   atrial_fibrillation                                              4082 non-null   int64  
 6   malignant_cancer                                                 4082 non-null   int64  
 7   chf                                       

In [64]:
#split X, y

X_train=cla_merged.drop(['los_bins'], axis=1)
y_train=cla_merged['los_bins']
# y_train=cla_merged.iloc[:,22:22]

X_test=cla_mergedT.drop(['los_bins'], axis=1)
y_test=cla_mergedT['los_bins']
# y_test=cla_mergedT.iloc[:,22:22]

X_hold=cla_mergedH.drop(['los_bins'], axis=1)
y_hold=cla_mergedH['los_bins']

In [65]:
#SMOTE
from imblearn.over_sampling import SMOTE

smote_sampler = SMOTE(random_state=10,sampling_strategy='minority')
X_smote, y_smote = smote_sampler.fit_resample(X_train, y_train)

In [66]:
print(y_train.value_counts())
print(y_smote.value_counts())

X_smote.describe()

0    11745
1     2536
Name: los_bins, dtype: int64
0    11745
1    11745
Name: los_bins, dtype: int64


,gender,admission_age,weight_admit,height,charlson_score,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,race_encode_African,race_encode_Asian,race_encode_Caucasian,race_encode_Hispanic,race_encode_Not Specified,race_encode_South American,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc,inr,pt,ptt
count,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000,23490.000000
mean,0.498510,66.660396,86.103064,169.672470,5.591613,0.198297,0.085355,0.266326,0.339676,0.081226,0.209068,0.252022,0.637974,0.371137,0.083780,0.086931,0.017497,0.563602,0.023542,0.096211,0.000341,0.027160,0.000128,0.017497,0.000426,0.454364,0.077905,0.044061,0.131247,0.081354,0.080162,0.191273,0.129630,0.004172,0.001831,0.014134,0.107237,0.082163,15.425045,22.713559,33.485358,8.301491,103.328344,1.896304,146.604761,138.172940,4.284004,31.152058,10.174801,29.959208,32.674373,91.527644,198.574554,3.421722,15.756237,12.100088,1.539374,16.623731,36.898446
std,0.500008,14.886958,26.427413,8.028729,2.753201,0.398726,0.279416,0.442046,0.473609,0.273188,0.406651,0.434183,0.480597,0.483119,0.277064,0.281739,0.131116,0.495949,0.151620,0.294887,0.018452,0.162554,0.011301,0.131116,0.020629,0.497924,0.268029,0.205236,0.337678,0.273383,0.271549,0.393312,0.335903,0.064457,0.042747,0.118045,0.309421,0.274618,3.752775,4.518751,23.716967,0.674572,6.084988,1.749473,56.154690,4.806215,0.577345,4.839791,1.612712,2.117272,1.288315,5.647886,94.195176,0.559436,1.898478,7.709639,0.610368,6.285846,12.641384
min,0.000000,18.024411,1.000000,122.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,5.000000,1.833333,4.050000,67.600000,0.100000,34.000000,105.333333,2.200000,13.600000,3.900000,16.700000,25.500000,61.750000,8.000000,1.360000,11.300000,0.100000,0.800000,8.700000,18.250000
25%,0.000000,57.333760,69.200000,166.335594,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.000000,20.000000,18.000000,7.900173,99.833333,0.900000,111.666667,135.670454,3.900000,27.914836,9.100000,28.950000,31.933333,88.500000,14

In [67]:
#normalise
num_cols = X_train.columns[X_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])

num_cols = X_test.columns[X_test.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_test[num_cols] = scaler.fit_transform(X_test[num_cols])

num_cols = X_hold.columns[X_hold.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
X_hold[num_cols] = scaler.fit_transform(X_hold[num_cols])

# Ridge

In [68]:
l2=RidgeClassifier()
l2.fit(X_train, y_train)

RidgeClassifier()

In [69]:
sel = SelectFromModel(l2)
sel.fit(X_train, y_train)

selected_feats=sel.get_feature_names_out()
print(selected_feats)
print(len(selected_feats))

['admission_age' 'height' 'stroke' 'admission_type_URGENT'
 'first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU)' 'bun'
 'calcium' 'hematocrit' 'hemoglobin' 'mch' 'mchc' 'mcv' 'rbc']
13


In [70]:
Best_l2 = RidgeClassifierCV(alphas=np.logspace(-6, 6, 13))

Best_l2.fit(X_train[selected_feats],y_train)

RidgeClassifierCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]))

In [71]:
Best_l2.alpha_

10.0

In [72]:
l2_pred = Best_l2.predict(X_test[selected_feats])
# l2_pp=Best_l2.predict_proba(X_test[selected_feats])

# l2_posProb = l2_pp[:,-1]

# rfc_f1 = f1_score(y_test,rfc_pred)
# rfc_auroc = roc_auc_score(y_test, rfc_pp[:,-1], multi_class="ovr")
# print(rfc_f1)
# print(rfc_auroc)
pd.crosstab(y_test, l2_pred)

col_0,0
los_bins,
0,3349
1,733


In [73]:
print(classification_report(y_test, l2_pred))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3349
           1       0.00      0.00      0.00       733

    accuracy                           0.82      4082
   macro avg       0.41      0.50      0.45      4082
weighted avg       0.67      0.82      0.74      4082



C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [74]:
l2_predH = Best_l2.predict(X_hold[selected_feats])
pd.crosstab(y_hold, l2_predH)

col_0,0
los_bins,
0,1654
1,387


In [75]:
print(classification_report(y_hold, l2_predH))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90      1654
           1       0.00      0.00      0.00       387

    accuracy                           0.81      2041
   macro avg       0.41      0.50      0.45      2041
weighted avg       0.66      0.81      0.73      2041



C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# select from model

In [76]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)

sel = SelectFromModel(rfc)
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(random_state=0))

In [77]:
selected_feats=sel.get_feature_names_out()
print(selected_feats)
print(len(selected_feats))

['admission_age' 'weight_admit' 'height' 'charlson_score' 'aniongap'
 'bicarbonate' 'bun' 'calcium' 'chloride' 'creatinine' 'glucose' 'sodium'
 'potassium' 'hematocrit' 'hemoglobin' 'mch' 'mchc' 'mcv' 'platelet' 'rbc'
 'rdw' 'wbc' 'inr' 'pt' 'ptt']
25


# RF Classifier (select from model)

In [78]:
rfc = RandomForestClassifier(criterion='entropy')

parameters = {'max_depth':np.arange(start=1,stop=10,step=1)}
stratifiedCV = StratifiedKFold(n_splits=5)
Best_rfc = GridSearchCV(
    rfc,
    param_grid=parameters,
    cv=stratifiedCV
)

Best_rfc.fit(X_smote[selected_feats],y_smote)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(criterion='entropy'),
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])})

In [79]:
Best_rfc.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=9)

In [80]:
rfc_pred = Best_rfc.predict(X_test[selected_feats])

In [81]:
np.count_nonzero(rfc_pred == 3)

0

In [82]:
#Test-RF with forward selected feature

rfc_pred = Best_rfc.predict(X_test[selected_feats])
# rfc_pp=Best_rfc.predict_proba(X_test[selected_feats])

# rfc_posProb = rfc_pp[:,-1]

# # rfc_f1 = f1_score(y_test,rfc_pred)
# rfc_auroc = roc_auc_score(y_test, rfc_pp[:,-1], multi_class="ovr")
# # print(rfc_f1)
# print(rfc_auroc)
pd.crosstab(y_test, rfc_pred)

col_0,0
los_bins,
0,3349
1,733


In [83]:
print(classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3349
           1       0.00      0.00      0.00       733

    accuracy                           0.82      4082
   macro avg       0.41      0.50      0.45      4082
weighted avg       0.67      0.82      0.74      4082



C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
rfc_predH = Best_rfc.predict(X_hold[selected_feats])
pd.crosstab(y_hold, rfc_predH)

col_0,0
los_bins,
0,1654
1,387


In [85]:
print(classification_report(y_hold, rfc_predH))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90      1654
           1       0.00      0.00      0.00       387

    accuracy                           0.81      2041
   macro avg       0.41      0.50      0.45      2041
weighted avg       0.66      0.81      0.73      2041



C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# XGB Classifier

In [86]:
xgbc =XGBC()

xgbc.fit(X_smote,y_smote)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [87]:
thresholds = np.sort(xgbc.feature_importances_)

In [88]:
thresholds

array([0.        , 0.0017788 , 0.00196858, 0.00204291, 0.00205782,
       0.00208395, 0.00213684, 0.00214734, 0.00218688, 0.00219091,
       0.00220055, 0.00220338, 0.00222879, 0.00223923, 0.00225338,
       0.00241562, 0.00242942, 0.00250895, 0.00251097, 0.0025132 ,
       0.00252376, 0.00252741, 0.00259543, 0.00264021, 0.00268582,
       0.00269925, 0.00279206, 0.00279716, 0.00312841, 0.0032554 ,
       0.00339425, 0.00383964, 0.0054279 , 0.00551593, 0.00580243,
       0.0069935 , 0.00952732, 0.01112183, 0.0166283 , 0.01698369,
       0.01825907, 0.01953921, 0.02400294, 0.02416825, 0.02750453,
       0.02909349, 0.03196073, 0.03362364, 0.03543483, 0.0393042 ,
       0.0399129 , 0.05209868, 0.05657068, 0.05827989, 0.05907246,
       0.06265762, 0.0652575 , 0.07115074, 0.09713141], dtype=float32)

In [89]:
for thresh in thresholds:
    selection = SelectFromModel(xgbc, threshold=thresh, prefit=True)
    select_X_smote = selection.transform(X_smote)
    selection_model = XGBC()
    selection_model.fit(select_X_smote, y_smote)
    select_X_test = selection.transform(X_test)
    predictions = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_smote.shape[1], accuracy*100.0))

C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.000, n=59, Accuracy: 81.92%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.002, n=58, Accuracy: 81.92%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


...

C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.063, n=4, Accuracy: 31.82%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Thresh=0.065, n=3, Accuracy: 30.55%
Thresh=0.071, n=2, Accuracy: 28.52%
Thresh=0.097, n=1, Accuracy: 20.24%


C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [90]:
selected=pd.DataFrame()
selected['col']=X_smote.columns
selected['rank']=xgbc.feature_importances_
selected.sort_values(by='rank', ascending=False,inplace=True)

top6=selected[:13]
selected_feats=top6['col'].to_list()
selected_feats

['race_encode_Hispanic',
 'race_encode_Not Specified',
 'admission_type_ELECTIVE',
 'race_encode_Asian',
 'admission_type_DIRECT EMER.',
 'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
 'admission_type_SURGICAL SAME DAY ADMISSION',
 'admission_type_URGENT',
 'first_careunit_Surgical Intensive Care Unit (SICU)',
 'race_encode_African',
 'first_careunit_Trauma SICU (TSICU)',
 'admission_type_OBSERVATION ADMIT',
 'height']

In [91]:

xgb = XGBC()

parameters = {
    'n_estimators':np.arange(start=2,stop=20,step=2),
    'max_depth':np.arange(start=2,stop=6,step=1),
    'learning_rate':np.arange(start=0.05,stop=0.4,step=0.05)
}
stratifiedCV = StratifiedKFold(n_splits=5)
Best_xgb = GridSearchCV(
    xgb,
    param_grid=parameters,
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1
)

Best_xgb.fit(X_smote[selected_feats],y_smote)

Fitting 5 folds for each of 252 candidates, totalling 1260 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35]),
                         'max_depth': array([2, 3, 4, 5]),
                         'n_estimators': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])},
             verbose=1)

In [92]:
Best_xgb.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.35000000000000003,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=18, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [93]:

xgb_pred = Best_xgb.predict(X_test[selected_feats])
# xgb_pp=Best_xgb.predict_proba(X_test[selected_feats])

# xgb_posProb = xgb_pp[:,-1]

# xgb_f1 = f1_score(y_test, xgb_pred)
# xgb_auroc = roc_auc_score(y_test, xgb_pp[:,-1])
# print(xgb_f1)
# print(xgb_auroc)
pd.crosstab(y_test, xgb_pred)

col_0,0,1
los_bins,,
0,1781,1568
1,361,372


In [94]:
print(classification_report(y_test, xgb_pred))

              precision    recall  f1-score   support

           0       0.83      0.53      0.65      3349
           1       0.19      0.51      0.28       733

    accuracy                           0.53      4082
   macro avg       0.51      0.52      0.46      4082
weighted avg       0.72      0.53      0.58      4082



In [95]:
xgb_predH = Best_xgb.predict(X_hold[selected_feats])
pd.crosstab(y_hold, xgb_predH)

col_0,0,1
los_bins,,
0,877,777
1,218,169


In [96]:
print(classification_report(y_hold, xgb_predH))

              precision    recall  f1-score   support

           0       0.80      0.53      0.64      1654
           1       0.18      0.44      0.25       387

    accuracy                           0.51      2041
   macro avg       0.49      0.48      0.45      2041
weighted avg       0.68      0.51      0.57      2041



# Support vector

In [99]:
from sklearn.svm import SVC

names=X_smote.columns.to_numpy()
names

array(['gender', 'admission_age', 'weight_admit', 'height',
       'charlson_score', 'atrial_fibrillation', 'malignant_cancer', 'chf',
       'ckd', 'cld', 'copd', 'diabetes', 'hypertension', 'ihd', 'stroke',
       'race_encode_African', 'race_encode_Asian',
       'race_encode_Caucasian', 'race_encode_Hispanic',
       'race_encode_Not Specified', 'race_encode_South American',
       'admission_type_DIRECT EMER.', 'admission_type_DIRECT OBSERVATION',
       'admission_type_ELECTIVE', 'admission_type_EU OBSERVATION',
       'admission_type_EW EMER.', 'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION',
       'admission_type_URGENT',
       'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
       'first_careunit_Coronary Care Unit (CCU)',
       'first_careunit_Medical Intensive Care Unit (MICU)',
       'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)',
       'first_careunit_Neuro Intermediate',
       'first_careunit_

In [100]:
sv = SVC(kernel='linear')
sv.fit(X_smote, y_smote)

coef =abs(sv.coef_[0])
coef

array([3.78177852e-02, 2.09368819e-03, 9.35871397e-04, 4.22647268e-03,
       1.76757808e-03, 4.05266228e-02, 8.89512744e-02, 9.45967578e-03,
       4.84426327e-02, 3.82332702e-02, 5.72907317e-03, 4.51398572e-02,
       7.25254232e-02, 3.92755971e-02, 8.50168212e-02, 2.91013765e+00,
       2.92014149e+00, 2.90803780e+00, 2.95227658e+00, 2.90886054e+00,
       2.85342183e+00, 2.94362643e+00, 2.22822554e+00, 3.00647342e+00,
       3.00000000e+00, 2.96034074e+00, 3.01644861e+00, 3.06281179e+00,
       2.86878363e+00, 2.99691408e+00, 3.09856060e+00, 3.02114349e+00,
       3.05337573e+00, 3.00000000e+00, 3.03082918e+00, 2.87606828e+00,
       2.96527292e+00, 2.95783570e+00, 4.67384159e-03, 2.77474519e-03,
       1.88730977e-03, 4.99108763e-02, 6.42009344e-04, 1.35600014e-02,
       8.04684836e-05, 1.02219654e-03, 1.19476206e-03, 7.34698538e-03,
       4.53474451e-02, 1.07076451e-01, 7.34069097e-02, 3.00200221e-02,
       6.26570452e-04, 1.90211061e-01, 9.00249431e-03, 3.17048726e-03,
      

In [101]:
sv_rank = pd.DataFrame({'features': names, 'coef': coef})
sv_rank.sort_values(by=['coef'], inplace=True, ascending=False)
sv_rank

,features,coef
30,first_careunit_Coronary Care Unit (CCU),3.098561
27,admission_type_SURGICAL SAME DAY ADMISSION,3.062812
32,first_careunit_Medical/Surgical Intensive Care...,3.053376
34,first_careunit_Neuro Stepdown,3.030829
31,first_careunit_Medical Intensive Care Unit (MICU),3.021143
26,admission_type_OBSERVATION ADMIT,3.016449
23,admission_type_ELECTIVE,3.006473
33,first_careunit_Neuro Intermediate,3.000000
24,admission_type_EU OBSERVATION,3.000000
29,first_careunit_Cardiac Vascular Intensive Care...,2.996914


In [102]:
top=sv_rank[:22]
selected_feats=top['features'].to_list()
selected_feats

['first_careunit_Coronary Care Unit (CCU)',
 'admission_type_SURGICAL SAME DAY ADMISSION',
 'first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)',
 'first_careunit_Neuro Stepdown',
 'first_careunit_Medical Intensive Care Unit (MICU)',
 'admission_type_OBSERVATION ADMIT',
 'admission_type_ELECTIVE',
 'first_careunit_Neuro Intermediate',
 'admission_type_EU OBSERVATION',
 'first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)',
 'first_careunit_Surgical Intensive Care Unit (SICU)',
 'admission_type_EW EMER.',
 'first_careunit_Trauma SICU (TSICU)',
 'race_encode_Hispanic',
 'admission_type_DIRECT EMER.',
 'race_encode_Asian',
 'race_encode_African',
 'race_encode_Not Specified',
 'race_encode_Caucasian',
 'first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU)',
 'admission_type_URGENT',
 'race_encode_South American']

In [104]:
sv = SVC(kernel='rbf')

parameters = {'C':np.arange(0.5,5,step=0.5)}
stratifiedCV = StratifiedKFold(n_splits=5)
sv_f0 = GridSearchCV(
    sv,
    param_grid=parameters,
    scoring='f1',
    cv=stratifiedCV,
    verbose=1,
    n_jobs=-1
)

sv_f0.fit(X_smote[selected_feats],y_smote)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SVC(), n_jobs=-1,
             param_grid={'C': array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5])},
             scoring='f1', verbose=1)

In [105]:
sv_f0.best_estimator_

SVC(C=0.5)

In [106]:
sv_f0.best_score_

0.7757032845996173

In [107]:
sv_f0_pred = sv_f0.predict(X_test[selected_feats])
pd.crosstab(y_test, sv_f0_pred)

col_0,0
los_bins,
0,3349
1,733


In [108]:
print(classification_report(y_test, sv_f0_pred))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90      3349
           1       0.00      0.00      0.00       733

    accuracy                           0.82      4082
   macro avg       0.41      0.50      0.45      4082
weighted avg       0.67      0.82      0.74      4082



C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
sv_f0_predH = sv_f0.predict(X_hold[selected_feats])
pd.crosstab(y_hold, sv_f0_predH)

col_0,0
los_bins,
0,1654
1,387


In [110]:
print(classification_report(y_hold, sv_f0_predH))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90      1654
           1       0.00      0.00      0.00       387

    accuracy                           0.81      2041
   macro avg       0.41      0.50      0.45      2041
weighted avg       0.66      0.81      0.73      2041



C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kawai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
